<a href="https://colab.research.google.com/github/pfuhr/InfoRet/blob/main/Pegasus_paraphrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.0 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece #library needed for Pegasus paraphraser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00


In [ ]:
#the following code cell is code copied from Huggingface to make use of the pegasus paraphraser
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving arguments.tsv to arguments.tsv


In [ ]:
import pandas as pd
from google.colab import files
#Loading argument data and organization into lists
arguments_df = pd.read_csv('arguments.tsv', delimiter='\t')
premises_list = arguments_df['Premise'].tolist()
conclusions_list = arguments_df['Conclusion'].tolist()
stance_list =  arguments_df['Stance'].tolist()
part_list = arguments_df['Part'].tolist()
argument_id_list = arguments_df['Argument ID'].tolist()
usage_list = arguments_df['Usage'].tolist()

#function that takes a list of texts (strings) and paraphrases each text individually
def paraphrase (texts):
  paraphrased_texts = []
  for text in texts:
    para = get_response(text, 1, 1)
    paraphrased_texts = paraphrased_texts + para
  return paraphrased_texts

#applying the function paraphrase to the premises and conclusions
para_premises_list = paraphrase(premises_list)
para_conclusions_list = paraphrase(conclusions_list)

#reconcattenating other lists with paraphrased lists and reconversion into tsv file
paraphrases = {'Argument ID': argument_id_list, 'Part': part_list, 'Usage': usage_list, 'Conclusion': para_conclusions_list, 'Stance': stance_list, 'Premise': para_premises_list}
paraphrases_df = pd.DataFrame(paraphrases)
paraphrases_df.to_csv('paraphrases.tsv', sep='\t', index=False)
files.download('paraphrases.tsv')



text: if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?
para: ['If it can be used to more easily capture wanted criminals, then it should be legal.']
text: we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.
para: ['Human cloning will cause huge issues when you have a bunch of the same humans acting the same.']
text: marriage is the ultimate commitment to someone, if people want it they should be allowrd
para: ['If people want to marry, they should be allowed.']
text: it provides a useful income for some people
para: ['It provides a source of income for some people.']
text: fast food should be banned because it is really bad for your health and is costly.
para: ['Fast food is bad for your health and costs a lot.']
text: sometimes economic sanctions are the only thing that will get the corrupt governments to take action
para: ['Economic sanctions are the 

KeyboardInterrupt: ignored